In [2]:
from collections import defaultdict, Counter
import numpy as np
import pandas as pd
from nltk.corpus import stopwords
from sklearn.metrics import log_loss
from sklearn.cross_validation import train_test_split
import argparse
import functools
# Our features
import feature_engineering
# XGBoost library
import xgboost as xgb
from xgboost import XGBClassifier

### Feature engineering 

In [3]:
initial_train, initial_test = feature_engineering.create_preprocessed_features()

In [5]:
tfidf_train, tfidf_test = feature_engineering.create_tfidf_features()

/home/alexattia/anaconda3/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:1089: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  if hasattr(X, 'dtype') and np.issubdtype(X.dtype, np.float):
/home/alexattia/Work/work_env/lib/python3.6/site-packages/numpy/core/fromnumeric.py:2957: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/home/alexattia/Work/work_env/lib/python3.6/site-packages/numpy/core/_methods.py:80: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


In [7]:
pr_train, pr_test = feature_engineering.create_pagerank_features()

In [9]:
X_train = pd.concat([initial_train, tfidf_train, pr_train], axis=1)
X_test = pd.concat([initial_test, tfidf_test, pr_test], axis=1)
X_train.shape, X_test.shape

((80100, 54), (20179, 54))

In [ ]:
X_train, X_valid, y_train, y_valid = train_test_split(X_train, y_train, test_size=0.1, random_state=4242)

In [ ]:
X_train = X_train.drop(["question1", "question2", "question1_nouns", "question2_nouns"],
                      axis=1)
X_valid = X_valid.drop(["question1", "question2", "question1_nouns", "question2_nouns"],
                      axis=1)
X_test = X_test.drop(["question1", "question2", "question1_nouns", "question2_nouns"],
                      axis=1)

### XGBoost 

#### Simple model

In [ ]:
# Finetuned parameters
params = {}
params['objective'] = 'binary:logistic'
params['eval_metric'] = 'logloss'
params['eta'] = 0.02
params['n_jobs'] = 5
params['max_depth'] = 6
params['subsample'] = 0.6
params['base_score'] = 0.2

d_train = xgb.DMatrix(X_train, label=y_train)
d_valid = xgb.DMatrix(X_valid, label=y_valid)

watchlist = [(d_train, 'train'), (d_valid, 'valid')]

In [ ]:
bst = xgb.train(params, d_train, 2500, watchlist, early_stopping_rounds=50, verbose_eval=200)
print(log_loss(y_valid, bst.predict(d_valid)))

In [ ]:
d_test = xgb.DMatrix(X_test)
p_test = bst.predict(d_test, ntree_limit=bst.best_ntree_limit)

#### Multi XGBoost for averaging

In [ ]:
df_tot = pd.DataFrame()
for i in range(9):
    np.random.seed(i+1)
    params = {}
    params['objective'] = 'binary:logistic'
    params['eval_metric'] = 'logloss'
    et = [.02,.025,.01,.015]
    params['eta'] = np.random.choice(et)
    params['n_jobs'] = 5
    depth = [4,5,6,7]
    params['max_depth'] = np.random.choice(depth)
    sub = [.5,.6,.7,.4]
    params['subsample'] = np.random.choice(sub)
    params['base_score'] = 0.2
    col = [1,.7]
    params['colsample_bytree'] = np.random.choice(col)

    d_train = xgb.DMatrix(X_train, label=y_train)
    d_valid = xgb.DMatrix(X_valid, label=y_valid)

    watchlist = [(d_train, 'train'), (d_valid, 'valid')]
    bst = xgb.train(params, d_train, 2500, watchlist, early_stopping_rounds=50, verbose_eval=2500)

    d_test = xgb.DMatrix(X_test)
    p_test = bst.predict(d_test, ntree_limit=bst.best_ntree_limit)
    df_tot[str('fold'+ str(i+1))] = p_test

#### LSTM

#### Averaging